In [1]:
# https://sci-hub.se/10.1002/sim.1203
# https://journals.sagepub.com/doi/pdf/10.1177/1536867x0900900206

# Dataset

In [2]:
from sksurv.metrics import concordance_index_censored
from lifelines.datasets import load_rossi

data = load_rossi()
data.dropna(inplace=True)
print(data.shape)
data.head()

(432, 9)


,week,arrest,fin,age,race,wexp,mar,paro,prio
0,20,1,0,27,1,0,0,1,3
1,17,1,0,18,1,0,0,1,8
2,25,1,0,19,0,1,0,1,13
3,52,0,1,23,1,1,1,1,1
4,52,0,0,19,0,1,0,1,3


In [3]:
event_col = "arrest"
duration_col = "week"

X = data.drop(columns=[event_col, duration_col])
X.shape

(432, 7)

In [4]:
from sklearn.model_selection import train_test_split

train_idx, test_idx = train_test_split(
    range(data.shape[0]), test_size=0.15, random_state=42, stratify=data[event_col]
)
data_train, data_test = data.iloc[train_idx], data.iloc[test_idx]
X_train, X_test = X.iloc[train_idx].values, X.iloc[test_idx].values
data_train.shape, data_test.shape

((367, 9), (65, 9))

# Pre-processing

In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train.shape, X_test.shape

((367, 7), (65, 7))

# Knots 

In [6]:
import numpy as np

log_t_test = np.log(data_test[duration_col])
log_t_train = np.log(data_train[duration_col])
min(log_t_train), max(log_t_train)

(0.0, 3.9512437185814275)

In [7]:
# Knot locations: Centiles of the distribution of **uncensored** log event times
# - Boundary knots: placed at the 0th and 100th centiles (min and max values)
# - Internal knots: internal knots are placed at the centiles between the min and max   
knots = np.percentile(log_t_train[data_train[event_col] == 1], [0, 25, 50, 75, 100])
min(knots), max(knots)

(0.0, 3.9512437185814275)

# Splines

In [8]:
def relu(x):
    return max(x, 0)


def spline_basis(x, k_j, k_min, k_max, derivative=False):
    """Computes the basis function S(x; k_j)."""
    # Scaling coefficient 
    s = (k_max - k_j) / (k_max - k_min)

    if derivative:
        # Derivative of the spline basis function
        return 3 * relu(x - k_j) ** 2 - 3 * s * relu(x - k_min) ** 2 - 3 * (1 - s) * relu(x - k_max) ** 2

    # Spline basis function 
    return relu(x - k_j) ** 3 - s * relu(x - k_min) ** 3 - (1 - s) * relu(x - k_max) ** 3


def spline_design_matrix(ln_t, knots):
    """Computes the spline function s(x; γ, k)."""
    # Boundary knots
    k_min, k_max = knots[0], knots[-1]
    # Construct basis functions over internal knots 
    basis = [spline_basis(ln_t, k_j, k_min, k_max) for k_j in knots[1:-1]]
    # Design matrix 
    return np.array([ln_t] + basis)


def spline_derivative_design_matrix(ln_t, knots):
    """Computes the spline function s(x; γ, k)."""
    # Boundary knots
    k_min, k_max = knots[0], knots[-1]
    # Construct basis functions over internal knots 
    basis = [spline_basis(ln_t, k_j, k_min, k_max, derivative=True) for k_j in knots[1:-1]]
    # Design matrix 
    return 1 / np.exp(ln_t) * np.array([1] + basis)


def create_splines(log_t, knots):

    D, dDdt = [], []
    for log_time in log_t:
        D.append(spline_design_matrix(log_time, knots))
        dDdt.append(spline_derivative_design_matrix(log_time, knots))
    
    # Cast to <numpy.ndarray>
    return np.array(D), np.array(dDdt)


D, D_prime = create_splines(log_t_train, knots)
D.shape, D_prime.shape

((367, 4), (367, 4))

# Parameter initialization 

In [9]:
delta_train = data_train[event_col].values[:, None]
delta_test = data_test[event_col].values[:, None]
delta_train.shape

(367, 1)

In [10]:
np.random.seed(42)
beta0 = np.random.random((1, X.shape[1]))
gamma0 = np.random.random((1, D.shape[1]))

# Fitting centralized model 

In [11]:
def gradient_gamma(gamma, beta, delta, D, D_prime, X):
    phi = D @ gamma.T + X_train @ beta.T

    ds_dt = D_prime @ gamma.T

    return delta * (D + D_prime / ds_dt) + np.exp(phi) * D


grad_gamma = gradient_gamma(gamma0, beta0, delta_train, D, D_prime, X_train)
grad_gamma[0] 

array([  3.65450862, -13.25046945,  -8.04299769,  -2.34127952])

In [12]:
def gradient_beta(gamma, beta, delta, D, D_prime, X):
    phi = D @ gamma.T + X_train @ beta.T

    return X * (delta + np.exp(phi))


grad_beta = gradient_beta(gamma0, beta0, delta_train, D, D_prime, X_train)
grad_beta[0] 

array([-0.98648792, -0.27112942,  0.37856055,  0.8791921 ,  2.78180058,
        0.78988578,  0.3479128 ])

In [13]:
def log_likelihood(gamma, beta, delta, D, D_prime, X):
    phi = D @ gamma.T + X_train @ beta.T

    ds_dt = np.clip(D_prime @ gamma.T, 1e-16, np.inf)

    return delta * (phi + np.log(ds_dt)) + np.exp(phi)


log_l = log_likelihood(gamma0, beta0, delta_train, D, D_prime, X_train)
log_l[0] 

array([-47.63031386])

In [33]:
def learning_rate_sched(epoch, init_lr=0.01, total_epochs=10):
    "Learning rate scheduler"
    return init_lr * 0.5 * (1 + np.cos(np.pi * epoch / total_epochs)) 


# Initialize parameters 
gamma = gamma0.copy()
beta = beta0.copy()

total_epochs = 10

for epoch in range(total_epochs):

    learning_rate = learning_rate_sched(epoch, init_lr=0.01, total_epochs=total_epochs)

    grad_gamma = gradient_gamma(gamma, beta, delta_train, D, D_prime, X_train).mean(axis=0)
    grad_beta = gradient_beta(gamma, beta, delta_train, D, D_prime, X_train).mean(axis=0)

    # Minimizing the negative log-likelihood
    gamma_new = gamma - learning_rate * grad_gamma.mean(axis=0)
    beta_new = beta - learning_rate * grad_beta.mean(axis=0)

    gamma = gamma_new 
    beta = beta_new 
    
    #if np.linalg.norm(gamma_new - gamma) / np.linalg.norm(gamma) > 1e-4:
    #    gamma = gamma_new 

    #if np.linalg.norm(beta_new - beta) / np.linalg.norm(beta) > 1e-4:
    #    beta = beta_new 

# Baseline
print(concordance_index_censored(delta_train.astype(bool).squeeze(), np.exp(log_t_train), (X_train @ beta0.T).squeeze()))
print(concordance_index_censored(delta_test.astype(bool).squeeze(), np.exp(log_t_test), (X_test @ beta0.T).squeeze()))

# Fitted model 
print(concordance_index_censored(delta_train.astype(bool).squeeze(), np.exp(log_t_train), (X_train @ beta.T).squeeze()))
print(concordance_index_censored(delta_test.astype(bool).squeeze(), np.exp(log_t_test), (X_test @ beta.T).squeeze()))

(0.41476774046744463, 12742, 17986, 35, 1080)
(0.2907465825446898, 276, 674, 1, 0)
(0.41454019438936385, 12735, 17993, 35, 1080)
(0.2886435331230284, 274, 676, 1, 0)


In [37]:
import tensorflow as tf

total_epochs = 10

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
gamma_var = tf.Variable(gamma0, dtype=tf.float32)
beta_var = tf.Variable(beta0, dtype=tf.float32)  

for epoch in range(total_epochs):
    
    grad_gamma = gradient_gamma(gamma, beta, delta_train, D, D_prime, X_train).sum(axis=0)
    grad_beta = gradient_beta(gamma, beta, delta_train, D, D_prime, X_train).sum(axis=0)

    gradients = [tf.cast(grad_beta, dtype=tf.float32), tf.cast(grad_gamma, dtype=tf.float32)]
    # Apply gradients to update gamma and beta
    optimizer.apply_gradients(zip(gradients, [beta_var, gamma_var]))
    gamma = gamma_var.numpy()
    beta = beta_var.numpy() 

# Baseline
print(concordance_index_censored(delta_train.astype(bool).squeeze(), np.exp(log_t_train), (X_train @ beta0.T).squeeze()))
print(concordance_index_censored(delta_test.astype(bool).squeeze(), np.exp(log_t_test), (X_test @ beta0.T).squeeze()))

# Fitted model 
print(concordance_index_censored(delta_train.astype(bool).squeeze(), np.exp(log_t_train), (X_train @ beta.T).squeeze()))
print(concordance_index_censored(delta_test.astype(bool).squeeze(), np.exp(log_t_test), (X_test @ beta.T).squeeze()))

(0.41476774046744463, 12742, 17986, 35, 1080)
(0.2907465825446898, 276, 674, 1, 0)
(0.40270779832916165, 12371, 18357, 35, 1080)
(0.2949526813880126, 280, 670, 1, 0)
